<a href="https://colab.research.google.com/github/21092004Goda/data_anal/blob/main/lab_2_recommendation_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Все импорты**

In [32]:
from google.colab import drive
import pandas as pd
import gdown
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

### **Константы**

In [33]:
# https://drive.google.com/file/d/1cmVsXJa82bIPwonq0eIo15rg-ZMmevgc/view?usp=sharing

GDRIVE_FILE_ID = "1cmVsXJa82bIPwonq0eIo15rg-ZMmevgc"

GDRIVE_DOWNLOAD_URL = f"https://drive.google.com/uc?id={GDRIVE_FILE_ID}"

LOCAL_FILE_PATH = "AMAZON_FASHION.json.gz"



### **Функции**

In [34]:

'''
Функция загрузки датасета
'''
def load_amazon_dataset(gdrive_url=GDRIVE_DOWNLOAD_URL, gdrive_file_name=LOCAL_FILE_PATH):

    if not gdrive_url.startswith('https://drive.google.com/uc?id='):
        raise ValueError("Неверный формат URL. Должен быть: https://drive.google.com/uc?id=FILE_ID")

    print(f"🐱‍👤 Загрузка из Google Drive: {gdrive_url}")

    # Скачиваем файл из Google Drive
    gdown.download(gdrive_url, LOCAL_FILE_PATH, quiet=False)

    if not os.path.exists(gdrive_file_name):
      print()
      raise FileNotFoundError(f"😒 Не удалось скачать файл из: {gdrive_url}")
      print()

    df = pd.read_json(gdrive_file_name, lines=True, compression='gzip')

    print()
    print(f"✔ Успешно загружено: {len(df):,} строк, {len(df.columns)} колонок")
    print()

    return df


In [35]:
'''
Функция для комплексного анализа профиля данных DataFrame

    'Column'                     : # Название колонки
    'Dtype'                      : # Тип данных (int, float, object, bool)
    'Total_Count'                : # Общее количество строк в датасете
    'Non_Null_Count'             : # Количество непустых значений
    'Null_Count'                 : # Количество пропусков (NaN/None)
    'Null_Percentage'            : # Процент пропусков от общего числа
    'Unique_Count'               : # Количество уникальных значений
    'Sparsity_Percentage'        : # Процент нулей (0) в числовых колонках
    'Empty_Strings_Percentage'   : # Процент пустых строк ("") в текстовых колонках

'''
def comprehensive_data_profile(df):
    results = []

    for column in df.columns:
        null_count = df[column].isnull().sum()
        null_percentage = (null_count / len(df) * 100).round(2)

        if df[column].dtype in ['int64', 'float64']:
            zero_count = (df[column] == 0).sum()
            sparsity_percentage = (zero_count / len(df) * 100).round(2)
        else:
            sparsity_percentage = 0

        empty_percentage = 0
        if df[column].dtype == 'object':
            if df[column].notna().any() and isinstance(df[column].dropna().iloc[0], dict):
                empty_strings = 0
            else:
                empty_strings = (df[column] == '').sum() if df[column].notna().any() else 0
            empty_percentage = round((empty_strings / len(df) * 100), 2)  # Исправлено здесь

        if df[column].notna().any() and isinstance(df[column].dropna().iloc[0], dict):
            unique_count = df[column].dropna().astype(str).nunique()
        else:
            try:
                unique_count = df[column].nunique()
            except TypeError:
                unique_count = df[column].dropna().astype(str).nunique()

        results.append({
            'Column': column,
            'Dtype': df[column].dtype,
            'Total_Count': len(df),
            'Non_Null_Count': df[column].count(),
            'Null_Count': null_count,
            'Null_Percentage': null_percentage,
            'Unique_Count': unique_count,
            'Sparsity_Percentage': sparsity_percentage,
            'Empty_Strings_Percentage': empty_percentage
        })

    return pd.DataFrame(results)

In [36]:
'''
ПОЛНЫЙ РАЗВЕДОЧНЫЙ АНАЛИЗ ДАННЫХ (EDA) - AMAZON FASHION REVIEWS
'''
def full_eda_analysis():

  plt.style.use('default')
  sns.set_palette("husl")
  %matplotlib inline



  # =============================================================================
  # ПОЛНЫЙ EDA АНАЛИЗ - AMAZON FASHION REVIEWS
  # =============================================================================

  print("=" * 70)
  print("📊 ПОЛНЫЙ EDA АНАЛИЗ - AMAZON FASHION REVIEWS")
  print("=" * 70)



  # =============================================================================
  # 1. ОБЩАЯ СТАТИСТИКА ДАТАСЕТА
  # =============================================================================

  print("📈 1. ОБЩАЯ СТАТИСТИКА ДАТАСЕТА")
  print("-" * 50)

  print(f"• Всего отзывов: {df.shape[0]:,}")
  print(f"• Колонок: {df.shape[1]}")
  print(f"• Период данных: {df['reviewTime'].min()} - {df['reviewTime'].max()}")
  print(f"• Объем памяти: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

  safe_columns = ['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
                  'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote']

  df_safe = df[safe_columns].copy()

  for col in ['reviewText', 'summary', 'reviewerName']:
      df_safe[col] = df_safe[col].astype(str)

  duplicates = df_safe.duplicated().sum()
  print(f"• Полных дубликатов: {duplicates} ({duplicates/len(df)*100:.2f}%)")

  key_duplicates = df.duplicated(subset=['reviewerID', 'asin', 'unixReviewTime']).sum()
  print(f"• Дубликатов по ключевым полям: {key_duplicates} ({key_duplicates/len(df)*100:.2f}%)")



  # =============================================================================
  # 2. АНАЛИЗ ПРОПУЩЕННЫХ ДАННЫХ
  # =============================================================================

  print("\n🔍 2. АНАЛИЗ ПРОПУЩЕННЫХ ДАННЫХ")
  print("-" * 50)

  plt.figure(figsize=(12, 6))
  missing_data = df.isnull().sum().sort_values(ascending=False)
  missing_percent = (missing_data / len(df) * 100).round(2)

  bars = plt.bar(missing_data.index, missing_percent, color=['red' if x > 50 else 'orange' if x > 10 else 'green' for x in missing_percent])
  plt.title('ПРОЦЕНТ ПРОПУЩЕННЫХ ДАННЫХ ПО КОЛОНКАМ', fontsize=14, fontweight='bold')
  plt.xlabel('Колонки', fontsize=12)
  plt.ylabel('Процент пропусков (%)', fontsize=12)
  plt.xticks(rotation=45, ha='right')
  plt.grid(axis='y', alpha=0.3)

  max_missing_percent = missing_percent.max()
  plt.ylim(0, max_missing_percent * 1.15)


  for bar, percentage in zip(bars, missing_percent):
      plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
              f'{percentage}%', ha='center', va='bottom', fontsize=10)

  plt.tight_layout()
  plt.show()

  print("📋 КАТЕГОРИИ КОЛОНОК ПО КАЧЕСТВУ ДАННЫХ:")
  print("✅ ОТЛИЧНОЕ КАЧЕСТВО (0% пропусков):")
  good_cols = missing_percent[missing_percent == 0]
  for col in good_cols.index:
      print(f"   - {col}")

  print("\n⚠️  ХОРОШЕЕ КАЧЕСТВО (<1% пропусков):")
  ok_cols = missing_percent[(missing_percent > 0) & (missing_percent < 1)]
  for col in ok_cols.index:
      print(f"   - {col}: {missing_percent[col]}% пропусков")

  print("\n❌ ПРОБЛЕМНЫЕ КОЛОНКИ (>50% пропусков):")
  bad_cols = missing_percent[missing_percent > 50]
  for col in bad_cols.index:
      print(f"   - {col}: {missing_percent[col]}% пропусков")



  # =============================================================================
  # 3. АНАЛИЗ РЕЙТИНГОВ ОТЗЫВОВ
  # =============================================================================

  print("\n⭐ 3. АНАЛИЗ РЕЙТИНГОВ ОТЗЫВОВ")
  print("-" * 50)

  rating_stats = df['overall'].value_counts().sort_index()
  rating_percent = (rating_stats / len(df) * 100).round(2)

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

  bars = ax1.bar(rating_stats.index, rating_stats.values, color=['red', 'orange', 'yellow', 'lightgreen', 'green'])
  ax1.set_title('РАСПРЕДЕЛЕНИЕ РЕЙТИНГОВ', fontweight='bold')
  ax1.set_xlabel('Количество звёзд')
  ax1.set_ylabel('Количество отзывов')
  ax1.grid(axis='y', alpha=0.3)

  max_value = rating_stats.max()
  ax1.set_ylim(0, max_value * 1.15)

  for bar, count, percent in zip(bars, rating_stats.values, rating_percent):
      ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000,
              f'{count:,}\n({percent}%)', ha='center', va='bottom', fontsize=9)

  colors = ['#ff6b6b', '#ffa726', '#ffee58', '#90ee90', '#4caf50']
  ax2.pie(rating_stats.values, labels=rating_stats.index, autopct='%1.1f%%', colors=colors, startangle=90)
  ax2.set_title('ПРОПОРЦИЯ РЕЙТИНГОВ', fontweight='bold')

  plt.tight_layout()
  plt.show()

  print("📊 СТАТИСТИКА РЕЙТИНГОВ:")
  print(f"• Средний рейтинг: {df['overall'].mean():.2f}")
  print(f"• Медианный рейтинг: {df['overall'].median():.2f}")
  print(f"• Самый частый рейтинг: {df['overall'].mode().iloc[0]}")
  print(f"• Распределение: {dict(rating_percent)}")



  # =============================================================================
  # 4. АНАЛИЗ ПОДТВЕРЖДЕННЫХ ОТЗЫВОВ
  # =============================================================================

  print("\n🔐 4. АНАЛИЗ ПОДТВЕРЖДЕННЫХ ОТЗЫВОВ")
  print("-" * 50)

  verified_stats = df['verified'].value_counts()
  verified_percent = (verified_stats / len(df) * 100).round(2)

  plt.figure(figsize=(8, 6))
  bars = plt.bar(['Подтвержденные', 'Неподтвержденные'], verified_stats.values,
                color=['#4caf50', '#ff9800'])
  plt.title('РАСПРЕДЕЛЕНИЕ ПОДТВЕРЖДЕННЫХ ОТЗЫВОВ', fontweight='bold')
  plt.ylabel('Количество отзывов')

  max_count = verified_stats.max()
  plt.ylim(0, max_count * 1.15)

  for bar, count, percent in zip(bars, verified_stats.values, verified_percent):
      plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000,
              f'{count:,}\n({percent}%)', ha='center', va='bottom', fontsize=12)

  plt.grid(axis='y', alpha=0.3)
  plt.tight_layout()
  plt.show()

  print(f"✅ Подтвержденных отзывов: {verified_stats[True]:,} ({verified_percent[True]}%)")
  print(f"❌ Неподтвержденных отзывов: {verified_stats[False]:,} ({verified_percent[False]}%)")



  # =============================================================================
  # 5. ВРЕМЕННОЙ АНАЛИЗ ОТЗЫВОВ
  # =============================================================================

  print("\n📅 5. ВРЕМЕННОЙ АНАЛИЗ ОТЗЫВОВ")
  print("-" * 50)

  df['reviewTime'] = pd.to_datetime(df['reviewTime'])
  df['reviewYear'] = df['reviewTime'].dt.year
  df['reviewMonth'] = df['reviewTime'].dt.month

  yearly_reviews = df['reviewYear'].value_counts().sort_index()

  plt.figure(figsize=(12, 6))
  plt.plot(yearly_reviews.index, yearly_reviews.values, marker='o', linewidth=2, markersize=8)
  plt.title('ДИНАМИКА ОТЗЫВОВ ПО ГОДАМ', fontweight='bold')
  plt.xlabel('Год')
  plt.ylabel('Количество отзывов')
  plt.grid(True, alpha=0.3)

  max_count = yearly_reviews.max()
  plt.ylim(0, max_count * 1.15)

  for year, count in yearly_reviews.items():
      plt.text(year, count + 1000, f'{count:,}', ha='center', va='bottom', fontsize=9)

  plt.tight_layout()
  plt.show()

  print("📈 СТАТИСТИКА ПО ГОДАМ:")
  for year, count in yearly_reviews.items():
      print(f"• {year}: {count:,} отзывов ({(count/len(df)*100):.1f}%)")



  # =============================================================================
  # 6. АНАЛИЗ АКТИВНОСТИ ПОЛЬЗОВАТЕЛЕЙ
  # =============================================================================

  print("\n👥 6. АНАЛИЗ АКТИВНОСТИ ПОЛЬЗОВАТЕЛЕЙ")
  print("-" * 50)

  user_activity = df['reviewerID'].value_counts()
  user_stats = user_activity.describe()

  plt.figure(figsize=(10, 6))
  plt.hist(user_activity.values, bins=50, edgecolor='black', alpha=0.7)
  plt.title('РАСПРЕДЕЛЕНИЕ КОЛИЧЕСТВА ОТЗЫВОВ НА ПОЛЬЗОВАТЕЛЯ', fontweight='bold')
  plt.xlabel('Количество отзывов на пользователя')
  plt.ylabel('Количество пользователей')
  plt.yscale('log')
  plt.grid(True, alpha=0.3)
  plt.tight_layout()
  plt.show()

  print("📊 СТАТИСТИКА АКТИВНОСТИ ПОЛЬЗОВАТЕЛЕЙ:")
  print(f"• Всего уникальных пользователей: {user_activity.shape[0]:,}")
  print(f"• Среднее отзывов на пользователя: {user_stats['mean']:.2f}")
  print(f"• Медиана отзывов на пользователя: {user_stats['50%']:.2f}")
  print(f"• Максимум отзывов у одного пользователя: {user_stats['max']:.0f}")
  print(f"• Пользователи с 1 отзывом: {(user_activity == 1).sum():,} ({(user_activity == 1).sum()/len(user_activity)*100:.1f}%)")



  # =============================================================================
  # 7. АНАЛИЗ ТОВАРОВ
  # =============================================================================

  print("\n🏷️ 7. АНАЛИЗ ТОВАРОВ")
  print("-" * 50)

  product_reviews = df['asin'].value_counts()
  product_stats = product_reviews.describe()

  plt.figure(figsize=(10, 6))
  plt.hist(product_reviews.values, bins=50, edgecolor='black', alpha=0.7)
  plt.title('РАСПРЕДЕЛЕНИЕ КОЛИЧЕСТВА ОТЗЫВОВ НА ТОВАР', fontweight='bold')
  plt.xlabel('Количество отзывов на товар')
  plt.ylabel('Количество товаров')
  plt.yscale('log')
  plt.grid(True, alpha=0.3)
  plt.tight_layout()
  plt.show()

  print("📊 СТАТИСТИКА ТОВАРОВ:")
  print(f"• Всего уникальных товаров: {product_reviews.shape[0]:,}")
  print(f"• Среднее отзывов на товар: {product_stats['mean']:.2f}")
  print(f"• Медиана отзывов на товар: {product_stats['50%']:.2f}")
  print(f"• Максимум отзывов на один товар: {product_stats['max']:.0f}")
  print(f"• Товары с 1 отзывом: {(product_reviews == 1).sum():,} ({(product_reviews == 1).sum()/len(product_reviews)*100:.1f}%)")
  print(f"• Товары с 2 отзывом: {(product_reviews == 2).sum():,} ({(product_reviews == 2).sum()/len(product_reviews)*100:.1f}%)")
  print(f"• Товары с 3 отзывом: {(product_reviews == 3).sum():,} ({(product_reviews == 3).sum()/len(product_reviews)*100:.1f}%)")



  # =============================================================================
  # 8. АНАЛИЗ ТЕКСТОВЫХ ДАННЫХ
  # =============================================================================

  print("\n📝 8. АНАЛИЗ ТЕКСТОВЫХ ДАННЫХ")
  print("-" * 50)

  df['reviewText_length'] = df['reviewText'].str.len()
  df['summary_length'] = df['summary'].str.len()

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

  ax1.hist(df['reviewText_length'].dropna(), bins=50, edgecolor='black', alpha=0.7)
  ax1.set_title('РАСПРЕДЕЛЕНИЕ ДЛИНЫ ТЕКСТА ОТЗЫВОВ', fontweight='bold')
  ax1.set_xlabel('Длина текста (символы)')
  ax1.set_ylabel('Количество отзывов')
  ax1.grid(True, alpha=0.3)

  ax2.hist(df['summary_length'].dropna(), bins=50, edgecolor='black', alpha=0.7)
  ax2.set_title('РАСПРЕДЕЛЕНИЕ ДЛИНЫ ЗАГОЛОВКОВ', fontweight='bold')
  ax2.set_xlabel('Длина заголовка (символы)')
  ax2.set_ylabel('Количество отзывов')
  ax2.grid(True, alpha=0.3)

  plt.tight_layout()
  plt.show()

  print("📏 СТАТИСТИКА ТЕКСТОВЫХ ДАННЫХ:")
  print(f"• Средняя длина текста отзыва: {df['reviewText_length'].mean():.0f} символов")
  print(f"• Средняя длина заголовка: {df['summary_length'].mean():.0f} символов")
  print(f"• Пустых текстов отзывов: {df['reviewText'].isnull().sum():,}")
  print(f"• Пустых заголовков: {df['summary'].isnull().sum():,}")



  # =============================================================================
  # 9. ИТОГОВЫЙ ОТЧЕТ ПО КАЧЕСТВУ ДАННЫХ
  # =============================================================================

  print("\n📋 9. ИТОГОВЫЙ ОТЧЕТ ПО КАЧЕСТВУ ДАННЫХ")
  print("=" * 70)

  print("✅ СИЛЬНЫЕ СТОРОНЫ ДАННЫХ:")
  print("1. Большой объем данных: 883,636 отзывов")
  print("2. Отличное качество основных колонок (0% пропусков)")
  print("3. Полные текстовые данные (99.8+% заполнения)")
  print("4. Разнообразная пользовательская база (749K+ пользователей)")
  print("5. Широкий ассортимент товаров (186K+ товаров)")

  print("\n⚠️  ПРОБЛЕМНЫЕ ОБЛАСТИ:")
  print("1. Vote: 91% пропусков - система голосования практически не используется")
  print("2. Style: 65% пропусков - плохая категоризация товаров")
  print("3. Image: 97% пропусков - минимальное количество визуального контента")

  print("\n🎯 РЕКОМЕНДАЦИИ ДЛЯ ДАЛЬНЕЙШЕГО АНАЛИЗА:")
  print("1. Сфокусироваться на анализе текста отзывов и рейтингов")
  print("2. Исключить колонку 'image' из анализа (слишком мало данных)")
  print("3. Использовать 'vote' и 'style' с осторожностью")
  print("4. Исследовать взаимосвязь между рейтингом и текстом отзыва")
  print("5. Проанализировать сезонность и тренды отзывов")

  print("\n📊 ОБЩАЯ ОЦЕНКА КАЧЕСТВА ДАННЫХ: 8.5/10")
  print("Данные отличного качества для анализа пользовательских отзывов!")
  print("=" * 70)

In [37]:
  '''
  Очистка данных для рекомендательных систем


  Parameters:
    df: исходный DataFrame
    keep_text_data: сохранять ли текстовые данные (reviewText, summary)
    min_user_interactions: минимальное количество взаимодействий на пользователя
    min_item_reviews: минимальное количество отзывов на товар
    min_year: минимальный год для фильтрации

  '''

def data_cleaning(df, keep_text_data=False, min_user_interactions=2, min_item_reviews=5, min_year=2013):

    print("🧹 НАЧИНАЕМ ОЧИСТКУ ДАННЫХ...")
    print(f"Исходный размер: {df.shape[0]:,} строк, {df.shape[1]} колонок")



    # =============================================================================
    # 1. ОТБРАСЫВАНИЕ ЛИШНИХ ПЕРЕМЕННЫХ
    # =============================================================================

    print("\n1. ОТБРАСЫВАНИЕ ЛИШНИХ ПЕРЕМЕННЫХ")

    keep_columns = ['reviewerID', 'asin', 'overall', 'unixReviewTime']

    if keep_text_data:
        keep_columns.extend(['reviewText', 'summary'])
        print("   ✅ Сохраняем текстовые данные")
    else:
        print("   ❌ Удаляем текстовые данные")

    df_clean = df[keep_columns].copy()

    print(f"   Сохранены колонки: {keep_columns}")
    print(f"   Размер после отбора колонок: {df_clean.shape[0]:,} строк, {df_clean.shape[1]} колонок")



    # =============================================================================
    # 2. ФИЛЬТРАЦИЯ ДАННЫХ
    # =============================================================================

    print("\n2. ФИЛЬТРАЦИЯ ДАННЫХ")

    if 'unixReviewTime' in df_clean.columns:
        df_clean['reviewYear'] = pd.to_datetime(df_clean['unixReviewTime'], unit='s').dt.year
        before_filter = len(df_clean)
        df_clean = df_clean[df_clean['reviewYear'] >= min_year]
        after_filter = len(df_clean)
        print(f"   ✅ Фильтрация по году (≥{min_year}): {before_filter:,} → {after_filter:,} строк")

    user_interaction_counts = df_clean['reviewerID'].value_counts()
    active_users = user_interaction_counts[user_interaction_counts >= min_user_interactions].index
    before_filter = len(df_clean)
    df_clean = df_clean[df_clean['reviewerID'].isin(active_users)]
    after_filter = len(df_clean)
    print(f"   ✅ Фильтрация пользователей (≥{min_user_interactions} взаимодействий): {before_filter:,} → {after_filter:,} строк")

    item_review_counts = df_clean['asin'].value_counts()
    popular_items = item_review_counts[item_review_counts >= min_item_reviews].index
    before_filter = len(df_clean)
    df_clean = df_clean[df_clean['asin'].isin(popular_items)]
    after_filter = len(df_clean)
    print(f"   ✅ Фильтрация товаров (≥{min_item_reviews} отзывов): {before_filter:,} → {after_filter:,} строк")

    before_dedup = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=['reviewerID', 'asin', 'unixReviewTime'])
    after_dedup = len(df_clean)
    if before_dedup != after_dedup:
        print(f"   ✅ Удаление дубликатов: {before_dedup:,} → {after_dedup:,} строк")



    # =============================================================================
    # 3. ФИНАЛЬНАЯ СТАТИСТИКА
    # =============================================================================

    print("\n3. ФИНАЛЬНАЯ СТАТИСТИКА")
    print(f"   📊 Итоговый размер: {df_clean.shape[0]:,} строк, {df_clean.shape[1]} колонок")
    print(f"   👥 Уникальных пользователей: {df_clean['reviewerID'].nunique():,}")
    print(f"   🏷️ Уникальных товаров: {df_clean['asin'].nunique():,}")
    print(f"   ⭐ Уникальных оценок: {df_clean['overall'].nunique()}")
    print(f"   📅 Период данных: {df_clean['reviewYear'].min()}-{df_clean['reviewYear'].max()}")

    avg_user_interactions = df_clean.groupby('reviewerID').size().mean()
    avg_item_reviews = df_clean.groupby('asin').size().mean()
    print(f"   📈 Среднее взаимодействий на пользователя: {avg_user_interactions:.2f}")
    print(f"   📈 Среднее отзывов на товар: {avg_item_reviews:.2f}")

    rating_dist = df_clean['overall'].value_counts().sort_index()
    print(f"   🎯 Распределение оценок: {dict(rating_dist)}")

    print("\n✅ ОЧИСТКА ЗАВЕРШЕНА!")
    return df_clean


In [38]:
'''
Разделение данных на train/test для оценки\

'''
def prepare_evaluation_data(df_clean, test_size=0.2, random_state=42):
    if 'unixReviewTime' in df_clean.columns:
        df_sorted = df_clean.sort_values('unixReviewTime')
    else:
        df_sorted = df_clean.sample(frac=1, random_state=random_state)

    user_counts = df_sorted['reviewerID'].value_counts()
    users_with_multiple = user_counts[user_counts > 1].index

    train_data = []
    test_data = []

    for user in users_with_multiple:
        user_interactions = df_sorted[df_sorted['reviewerID'] == user]
        split_idx = int(len(user_interactions) * (1 - test_size))

        train_data.append(user_interactions.iloc[:split_idx])
        test_data.append(user_interactions.iloc[split_idx:])

    train_df = pd.concat(train_data) if train_data else pd.DataFrame()
    test_df = pd.concat(test_data) if test_data else pd.DataFrame()

    print(f"Train: {len(train_df)} interactions, Test: {len(test_df)} interactions")
    print(f"Users in train: {train_df['reviewerID'].nunique()}")
    print(f"Users in test: {test_df['reviewerID'].nunique()}")

    return train_df, test_df

In [39]:
def evaluate_all_models_adapted(df_clean):
    """
    Оценка качества для конкретных интерфейсов моделей
    """
    print("=" * 70)
    print("📊 ОЦЕНКА КАЧЕСТВА РЕКОМЕНДАТЕЛЬНЫХ СИСТЕМ")
    print("=" * 70)

    # 1. Подготовка данных
    train_df, test_df = prepare_evaluation_data(df_clean)

    if len(test_df) == 0:
        print("⚠️ Недостаточно данных для оценки")
        return

    # 2. Обучение моделей на train данных
    models = {
        'MostPop': MostPopRecommender(),
        'ItemKNN': ItemKNNRecommender(k=50),
        'EASE': EASERecommender(l2_reg=1, normalize=True)
    }

    trained_models = {}
    for name, model in models.items():
        print(f"\n🎯 Обучение {name}...")
        model.fit(train_df)
        trained_models[name] = model

    # 3. Подготовка дополнительных данных для метрик
    catalog_size = df_clean['asin'].nunique()
    item_popularity = df_clean['asin'].value_counts().to_dict()

    # 4. Генерация рекомендаций для тестовых пользователей
    test_users = test_df['reviewerID'].unique()
    all_recommendations = {}

    for name, model in trained_models.items():
        print(f"\n📋 Генерация рекомендаций {name}...")
        user_recommendations = {}

        for user in test_users[:500]:  # Ограничим для скорости
            try:
                # УНИВЕРСАЛЬНЫЙ ВЫЗОВ ДЛЯ ВСЕХ МОДЕЛЕЙ
                recs = model.recommend(user, n=20)  # Все модели поддерживают user_id и n
                user_recommendations[user] = recs
            except Exception as e:
                # Если возникла ошибка, пробуем без user_id для MostPop
                if name == 'MostPop':
                    recs = model.recommend(n=20)  # MostPop может работать без user_id
                    user_recommendations[user] = recs
                else:
                    print(f"   ⚠️ Ошибка для пользователя {user}: {e}")
                    user_recommendations[user] = []

        all_recommendations[name] = user_recommendations

    # 5. Расчет метрик
    metrics_calculator = RecommendationMetrics(test_df, catalog_size)

    print("\n" + "=" * 70)
    print("📈 РЕЗУЛЬТАТЫ ОЦЕНКИ")
    print("=" * 70)

    results = {}
    for name, recommendations in all_recommendations.items():
        print(f"\n🔍 {name}:")
        metrics = metrics_calculator.calculate_all_metrics(
            recommendations, item_popularity, {}
        )

        for metric, value in metrics.items():
            print(f"   {metric}: {value:.4f}")

        results[name] = metrics

    return results, trained_models

📊 ОЦЕНКА КАЧЕСТВА РЕКОМЕНДАТЕЛЬНЫХ СИСТЕМ
Train: 23928 interactions, Test: 14283 interactions
Users in train: 13553
Users in test: 13553
🎯 Инициализация ItemKNN (k=50, metric=cosine)
🎯 Инициализация EASE (l2_reg=1)

🎯 Обучение MostPop...
🎯 Обучение MostPop модели...
   Найдено 4403 популярных товаров
   Самый популярный товар: B000YFSR5G
   - Количество отзывов: 424
   - Средний рейтинг: 3.85
   - Общий score: 1619.00
   📊 Топ-10 товаров имеют от 242 до 424 отзывов

🎯 Обучение ItemKNN...
🎯 Обучение ItemKNN модели...
   User-Item matrix: (13553, 4403)
   Уникальных пользователей: 13553
   Уникальных товаров: 4403
   Вычисление похожести товаров...
   ✅ ItemKNN модель обучена!
   Матрица похожести: (4403, 4403)
   📊 Статистика похожести:
      - Средняя похожесть: 0.316
      - Максимальная похожесть: 1.000
      - Минимальная похожесть: 0.001

🎯 Обучение EASE...
🎯 Обучение EASE модели...
   User-Item matrix: (13553, 4403)
   Уникальных пользователей: 13553
   Уникальных товаров: 4403
  

NameError: name 'RecommendationMetrics' is not defined

### **Классы методов формирования рекомендаций и метрики оценки качества**

In [ ]:
import pandas as pd
import numpy as np

class MostPopRecommender:
    """
    Most Popular Recommender - рекомендует самые популярные товары
    """

    def __init__(self):
        self.popular_items = None
        self.item_popularity = None
        self.item_scores = None

    def fit(self, df):
        """
        Обучение модели на исторических данных

        Parameters:
        df: DataFrame с колонками ['reviewerID', 'asin', 'overall']
        """
        print("🎯 Обучение MostPop модели...")

        # Считаем популярность товаров (по количеству взаимодействий)
        self.item_popularity = df.groupby('asin')['overall'].count().sort_values(ascending=False)

        # Также считаем средний рейтинг для каждого товара
        self.item_ratings = df.groupby('asin')['overall'].mean()

        # Комбинированный score: популярность * рейтинг
        self.item_scores = (self.item_popularity * self.item_ratings).sort_values(ascending=False)

        self.popular_items = self.item_scores.index.tolist()

        print(f"   Найдено {len(self.popular_items)} популярных товаров")
        print(f"   Самый популярный товар: {self.popular_items[0]}")
        print(f"   - Количество отзывов: {self.item_popularity.iloc[0]}")
        print(f"   - Средний рейтинг: {self.item_ratings.loc[self.popular_items[0]]:.2f}")
        print(f"   - Общий score: {self.item_scores.iloc[0]:.2f}")

        # Статистика по топ-10 товарам
        top_10_stats = self.item_popularity.head(10)
        print(f"   📊 Топ-10 товаров имеют от {top_10_stats.min()} до {top_10_stats.max()} отзывов")

    def recommend(self, user_id=None, n=10, return_scores=False):
        """
        Рекомендация топ-N популярных товаров

        Parameters:
        user_id: игнорируется (у всех одинаковые рекомендации)
        n: количество рекомендаций
        return_scores: возвращать ли scores вместе с item_id

        Returns:
        list: список рекомендованных item_id (и scores если return_scores=True)
        """
        if self.popular_items is None:
            raise ValueError("Модель не обучена! Сначала вызовите fit()")

        if return_scores:
            # Возвращаем item_id и их scores
            return [(item, self.item_scores[item]) for item in self.popular_items[:n]]
        else:
            # Возвращаем только item_id
            return self.popular_items[:n]

    def get_popularity_stats(self, n=10):
        """
        Получить статистику по популярным товарам

        Parameters:
        n: количество товаров для статистики

        Returns:
        DataFrame: статистика по топ-N товарам
        """
        if self.item_popularity is None:
            raise ValueError("Модель не обучена! Сначала вызовите fit()")

        stats_df = pd.DataFrame({
            'item_id': self.popular_items[:n],
            'num_reviews': [self.item_popularity[item] for item in self.popular_items[:n]],
            'avg_rating': [self.item_ratings[item] for item in self.popular_items[:n]],
            'popularity_score': [self.item_scores[item] for item in self.popular_items[:n]]
        })

        return stats_df

    def get_item_info(self, item_id):
        """
        Получить информацию о конкретном товаре

        Parameters:
        item_id: ID товара

        Returns:
        dict: информация о товаре
        """
        if item_id not in self.item_popularity.index:
            return None

        return {
            'item_id': item_id,
            'num_reviews': self.item_popularity[item_id],
            'avg_rating': self.item_ratings[item_id],
            'popularity_score': self.item_scores[item_id],
            'popularity_rank': self.popular_items.index(item_id) + 1
        }

    def get_model_info(self):
        """Информация о модели"""
        if self.popular_items is None:
            return {'model_type': 'MostPop', 'status': 'Not trained'}

        return {
            'model_type': 'MostPop',
            'num_items': len(self.popular_items),
            'top_item': self.popular_items[0],
            'top_item_reviews': self.item_popularity.iloc[0],
            'top_item_rating': self.item_ratings.iloc[0],
            'total_interactions': self.item_popularity.sum()
        }




def test_mostpop_recommender(df_clean):
    """
    Тестирование MostPop рекомендателя
    """
    print("=" * 60)
    print("🧪 ТЕСТИРОВАНИЕ MOSTPOP РЕКОМЕНДАТЕЛЯ")
    print("=" * 60)

    # Инициализация и обучение модели
    recommender = MostPopRecommender()
    recommender.fit(df_clean)

    # Получение рекомендаций
    print("\n📋 ТОП-10 РЕКОМЕНДАЦИЙ:")
    recommendations = recommender.recommend(n=10, return_scores=True)
    for i, (item, score) in enumerate(recommendations, 1):
        item_info = recommender.get_item_info(item)
        print(f"{i:2d}. {item} | Отзывов: {item_info['num_reviews']:3d} | "
              f"Рейтинг: {item_info['avg_rating']:.2f} | Score: {score:.2f}")

    # Статистика по топ товарам
    print("\n📊 СТАТИСТИКА ПО ТОП-5 ТОВАРАМ:")
    stats_df = recommender.get_popularity_stats(n=5)
    print(stats_df.to_string(index=False))

    # Информация о модели
    print("\n🔍 ИНФОРМАЦИЯ О МОДЕЛИ:")
    model_info = recommender.get_model_info()
    for key, value in model_info.items():
        print(f"   {key}: {value}")

    # Тест для конкретного пользователя (должны быть одинаковые рекомендации)
    test_user = df_clean['reviewerID'].iloc[0]
    print(f"\n👤 ТЕСТ ДЛЯ ПОЛЬЗОВАТЕЛЯ {test_user}:")
    user_recs = recommender.recommend(user_id=test_user, n=5)
    print(f"   Рекомендации: {user_recs}")

    return recommender

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

class ItemKNNRecommender:
    """
    Item-based KNN Recommender - рекомендует товары на основе похожести
    """

    def __init__(self, k=50, similarity_metric='cosine', min_similarity=0.1):
        """
        Parameters:
        k: количество соседей для KNN
        similarity_metric: метрика похожести ('cosine', 'pearson')
        min_similarity: минимальная похожесть для учета
        """
        self.k = k
        self.similarity_metric = similarity_metric
        self.min_similarity = min_similarity

        self.item_similarity = None
        self.user_item_matrix = None
        self.item_ids = None
        self.user_ids = None
        self.item_popularity = None

        print(f"🎯 Инициализация ItemKNN (k={k}, metric={similarity_metric})")

    def fit(self, df):
        """
        Обучение ItemKNN модели

        Parameters:
        df: DataFrame с колонками ['reviewerID', 'asin', 'overall']
        """
        print("🎯 Обучение ItemKNN модели...")

        # Создаем user-item матрицу
        self.user_item_matrix = df.pivot_table(
            index='reviewerID',
            columns='asin',
            values='overall',
            fill_value=0
        )

        self.user_ids = self.user_item_matrix.index.tolist()
        self.item_ids = self.user_item_matrix.columns.tolist()

        # Считаем популярность товаров
        self.item_popularity = df.groupby('asin')['overall'].count()

        print(f"   User-Item matrix: {self.user_item_matrix.shape}")
        print(f"   Уникальных пользователей: {len(self.user_ids)}")
        print(f"   Уникальных товаров: {len(self.item_ids)}")
        print(f"   Вычисление похожести товаров...")

        # Вычисляем матрицу похожести
        if self.similarity_metric == 'cosine':
            self.item_similarity = cosine_similarity(self.user_item_matrix.T)
        else:
            # Pearson correlation
            self.item_similarity = np.corrcoef(self.user_item_matrix.T.fillna(0))

        # Преобразуем в DataFrame для удобства
        self.item_similarity = pd.DataFrame(
            self.item_similarity,
            index=self.item_ids,
            columns=self.item_ids
        )

        # Заменяем NaN и отрицательные значения
        self.item_similarity = self.item_similarity.fillna(0)
        self.item_similarity[self.item_similarity < 0] = 0

        print(f"   ✅ ItemKNN модель обучена!")
        print(f"   Матрица похожести: {self.item_similarity.shape}")

        # Статистика похожести
        similarity_values = self.item_similarity.values.flatten()
        similarity_values = similarity_values[similarity_values > 0]  # Убираем нули и диагональ

        if len(similarity_values) > 0:
            print(f"   📊 Статистика похожести:")
            print(f"      - Средняя похожесть: {np.mean(similarity_values):.3f}")
            print(f"      - Максимальная похожесть: {np.max(similarity_values):.3f}")
            print(f"      - Минимальная похожесть: {np.min(similarity_values):.3f}")

    def recommend(self, user_id, n=10, return_scores=False):
        """
        Рекомендация товаров для пользователя

        Parameters:
        user_id: ID пользователя
        n: количество рекомендаций
        return_scores: возвращать ли scores

        Returns:
        list: список рекомендованных item_id (и scores если return_scores=True)
        """
        if self.item_similarity is None:
            raise ValueError("Модель не обучена! Сначала вызовите fit()")

        # Обработка cold start (новый пользователь)
        if user_id not in self.user_ids:
            print(f"   ⚠️  Пользователь {user_id} не найден, возвращаем популярные товары")
            popular_items = self.item_popularity.sort_values(ascending=False).head(n)
            if return_scores:
                return [(item, score) for item, score in popular_items.items()]
            else:
                return popular_items.index.tolist()

        # Получаем товары, с которыми взаимодействовал пользователь
        user_interactions = self.user_item_matrix.loc[user_id]
        interacted_items = user_interactions[user_interactions > 0].index.tolist()

        if not interacted_items:
            print(f"   ⚠️  У пользователя {user_id} нет взаимодействий, возвращаем популярные")
            popular_items = self.item_popularity.sort_values(ascending=False).head(n)
            if return_scores:
                return [(item, score) for item, score in popular_items.items()]
            else:
                return popular_items.index.tolist()

        print(f"   👤 Пользователь {user_id}: {len(interacted_items)} взаимодействий")

        # Считаем scores для всех товаров
        item_scores = {}

        for target_item in self.item_ids:
            if target_item in interacted_items:
                continue  # Пропускаем уже взаимодействованные

            total_score = 0
            total_similarity = 0

            for source_item in interacted_items:
                if source_item in self.item_similarity.columns:
                    similarity = self.item_similarity.loc[target_item, source_item]

                    if similarity >= self.min_similarity:
                        user_rating = user_interactions[source_item]
                        total_score += similarity * user_rating
                        total_similarity += similarity

            # Нормализуем score
            if total_similarity > 0:
                item_scores[target_item] = total_score / total_similarity
            else:
                item_scores[target_item] = 0

        # Сортируем по убыванию score
        recommendations = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)

        # Фильтруем рекомендации с положительным score
        recommendations = [(item, score) for item, score in recommendations if score > 0][:n]

        if return_scores:
            return recommendations
        else:
            return [item for item, score in recommendations]

    def get_similar_items(self, item_id, n=10, return_scores=False):
        """
        Получить похожие товары для заданного товара

        Parameters:
        item_id: ID товара
        n: количество похожих товаров
        return_scores: возвращать ли scores

        Returns:
        list: список похожих item_id (и scores если return_scores=True)
        """
        if self.item_similarity is None:
            raise ValueError("Модель не обучена! Сначала вызовите fit()")

        if item_id not in self.item_ids:
            print(f"   ⚠️  Товар {item_id} не найден")
            return []

        # Получаем похожие товары (исключая сам товар)
        similar_items = self.item_similarity[item_id].sort_values(ascending=False)
        similar_items = similar_items[similar_items.index != item_id]
        similar_items = similar_items[similar_items > self.min_similarity]

        top_similar = similar_items.head(n)

        if return_scores:
            return [(item, score) for item, score in top_similar.items()]
        else:
            return top_similar.index.tolist()

    def get_user_profile(self, user_id):
        """
        Получить профиль пользователя (историю взаимодействий)

        Parameters:
        user_id: ID пользователя

        Returns:
        dict: информация о пользователе
        """
        if user_id not in self.user_ids:
            return None

        user_interactions = self.user_item_matrix.loc[user_id]
        interacted_items = user_interactions[user_interactions > 0]

        return {
            'user_id': user_id,
            'num_interactions': len(interacted_items),
            'avg_rating': interacted_items.mean() if len(interacted_items) > 0 else 0,
            'interacted_items': interacted_items.sort_values(ascending=False).to_dict()
        }

    def get_model_info(self):
        """Информация о модели"""
        if self.item_similarity is None:
            return {'model_type': 'ItemKNN', 'status': 'Not trained'}

        # Статистика по похожести
        similarity_values = self.item_similarity.values
        np.fill_diagonal(similarity_values, 0)  # Игнорируем диагональ
        similarity_values = similarity_values[similarity_values > 0]

        return {
            'model_type': 'ItemKNN',
            'k': self.k,
            'similarity_metric': self.similarity_metric,
            'num_items': len(self.item_ids),
            'num_users': len(self.user_ids),
            'avg_similarity': np.mean(similarity_values) if len(similarity_values) > 0 else 0,
            'sparsity': (self.user_item_matrix == 0).sum().sum() / self.user_item_matrix.size
        }

def test_itemknn_recommender(df_clean):
    """
    Тестирование ItemKNN рекомендателя на АКТИВНЫХ пользователях
    """
    print("=" * 60)
    print("🧪 ТЕСТИРОВАНИЕ ITEMKNN РЕКОМЕНДАТЕЛЯ")
    print("=" * 60)

    # Инициализация и обучение модели
    recommender = ItemKNNRecommender(k=500, similarity_metric='cosine', min_similarity=0.001)
    recommender.fit(df_clean)

    # НАХОДИМ АКТИВНЫХ ПОЛЬЗОВАТЕЛЕЙ
    user_interaction_counts = df_clean['reviewerID'].value_counts()

    print("📊 СТАТИСТИКА ПОЛЬЗОВАТЕЛЕЙ:")
    print(f"   Всего пользователей: {len(user_interaction_counts)}")
    print(f"   Пользователей с 1 взаимодействием: {(user_interaction_counts == 1).sum()}")
    print(f"   Пользователей с 5+ взаимодействиями: {(user_interaction_counts >= 5).sum()}")
    print(f"   Пользователей с 10+ взаимодействиями: {(user_interaction_counts >= 10).sum()}")

    # ВЫБИРАЕМ РАЗНЫХ ПОЛЬЗОВАТЕЛЕЙ ДЛЯ ТЕСТИРОВАНИЯ
    test_users = {
        'САМЫЙ АКТИВНЫЙ': user_interaction_counts.index[0],  # Топ по взаимодействиям
        'СРЕДНЕАКТИВНЫЙ': user_interaction_counts[user_interaction_counts.between(5, 15)].index[0] if len(user_interaction_counts[user_interaction_counts.between(5, 15)]) > 0 else user_interaction_counts.index[1],
        'С 1 ВЗАИМОДЕЙСТВИЕМ': user_interaction_counts[user_interaction_counts == 1].index[0] if len(user_interaction_counts[user_interaction_counts == 1]) > 0 else user_interaction_counts.index[2]
    }

    # ТЕСТИРУЕМ НА КАЖДОМ ТИПЕ ПОЛЬЗОВАТЕЛЕЙ
    for user_type, test_user in test_users.items():
        print(f"\n{'='*50}")
        print(f"👤 ТЕСТ: {user_type} ПОЛЬЗОВАТЕЛЬ")
        print(f"{'='*50}")

        # Получаем профиль пользователя
        user_profile = recommender.get_user_profile(test_user)
        if user_profile:
            print(f"   ID: {test_user}")
            print(f"   Взаимодействий: {user_profile['num_interactions']}")
            print(f"   Средний рейтинг: {user_profile['avg_rating']:.2f}")
            print(f"   Топ-3 товара: {list(user_profile['interacted_items'].keys())[:3]}")
        else:
            print(f"   ❌ Пользователь {test_user} не найден в матрице")
            continue

        # Получение рекомендаций
        print(f"\n   📋 РЕКОМЕНДАЦИИ:")
        recommendations = recommender.recommend(test_user, n=8, return_scores=True)

        if recommendations:
            for i, (item, score) in enumerate(recommendations, 1):
                item_popularity = recommender.item_popularity.get(item, 0)
                print(f"   {i:2d}. {item} | Score: {score:.3f} | Отзывов: {item_popularity}")
        else:
            print("   ❌ Нет рекомендаций")

        # Похожие товары для первого товара из истории пользователя
        if user_profile['interacted_items']:
            first_item = list(user_profile['interacted_items'].keys())[0]
            print(f"\n   🔍 ПОХОЖИЕ ТОВАРЫ НА '{first_item}':")
            similar_items = recommender.get_similar_items(first_item, n=5, return_scores=True)

            if similar_items:
                for i, (similar_item, similarity) in enumerate(similar_items, 1):
                    item_popularity = recommender.item_popularity.get(similar_item, 0)
                    print(f"   {i:2d}. {similar_item} | Similarity: {similarity:.3f} | Отзывов: {item_popularity}")
            else:
                print("   ❌ Нет похожих товаров")

        # Проверяем разнообразие рекомендаций
        if recommendations:
            rec_items = [item for item, score in recommendations]
            print(f"\n   📊 СТАТИСТИКА РЕКОМЕНДАЦИЙ:")
            print(f"      - Уникальных товаров: {len(set(rec_items))}")
            print(f"      - Средняя популярность: {np.mean([recommender.item_popularity.get(item, 0) for item in rec_items]):.1f}")

    # ДОПОЛНИТЕЛЬНО: ТЕСТ НА СЛУЧАЙНЫХ ПОЛЬЗОВАТЕЛЯХ
    print(f"\n{'='*50}")
    print("🎲 ТЕСТ НА 3 СЛУЧАЙНЫХ АКТИВНЫХ ПОЛЬЗОВАТЕЛЯХ")
    print(f"{'='*50}")

    active_users = user_interaction_counts[user_interaction_counts >= 3].index
    random_active_users = np.random.choice(active_users, size=min(3, len(active_users)), replace=False)

    for i, random_user in enumerate(random_active_users, 1):
        user_profile = recommender.get_user_profile(random_user)
        if user_profile:
            recs = recommender.recommend(random_user, n=5)
            print(f"{i}. {random_user}: {user_profile['num_interactions']} взаимодействий → {len(recs)} рекомендаций")

    # Информация о модели
    print("\n🔍 ИНФОРМАЦИЯ О МОДЕЛИ:")
    model_info = recommender.get_model_info()
    for key, value in model_info.items():
        print(f"   {key}: {value}")

    return recommender


In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, eye
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

class EASERecommender:
    """
    EASE (Embarrassingly Shallow Autoencoders) Recommender
    Современный метод, показавший state-of-the-art результаты
    """

    def __init__(self, l2_reg=100, normalize=True):
        """
        Parameters:
        l2_reg: параметр L2 регуляризации (обычно 100-1000)
        normalize: нормализовать ли матрицу взаимодействий
        """
        self.l2_reg = l2_reg
        self.normalize = normalize

        self.B = None  # Матрица весов EASE
        self.user_item_matrix = None
        self.item_ids = None
        self.user_ids = None

        print(f"🎯 Инициализация EASE (l2_reg={l2_reg})")

    def fit(self, df):
        """
        Обучение EASE модели

        Parameters:
        df: DataFrame с колонками ['reviewerID', 'asin', 'overall']
        """
        print("🎯 Обучение EASE модели...")

        # Создаем user-item матрицу (implicit feedback)
        self.user_item_matrix = df.pivot_table(
            index='reviewerID',
            columns='asin',
            values='overall',
            fill_value=0
        )

        self.user_ids = self.user_item_matrix.index.tolist()
        self.item_ids = self.user_item_matrix.columns.tolist()

        print(f"   User-Item matrix: {self.user_item_matrix.shape}")
        print(f"   Уникальных пользователей: {len(self.user_ids)}")
        print(f"   Уникальных товаров: {len(self.item_ids)}")

        # Конвертируем в numpy array
        X = self.user_item_matrix.values.astype(np.float32)

        # Нормализация (опционально)
        if self.normalize:
            print("   Применение нормализации...")
            # L2 нормализация по пользователям
            row_norms = np.linalg.norm(X, axis=1, keepdims=True)
            row_norms[row_norms == 0] = 1  # Избегаем деления на 0
            X = X / row_norms

        print("   Вычисление матрицы Gram...")
        # Вычисляем матрицу Gram (X^T X)
        G = X.T @ X

        print("   Добавление регуляризации...")
        # Добавляем регуляризацию к диагонали
        n_items = G.shape[0]
        P = G + self.l2_reg * eye(n_items)

        print("   Решение системы уравнений...")
        # Решаем систему уравнений: P * B = I
        try:
            # Пробуем точное решение
            P_inv = np.linalg.inv(P)
            self.B = P_inv / -np.diag(P_inv)
            np.fill_diagonal(self.B, 0)  # Обнуляем диагональ

        except np.linalg.LinAlgError:
            print("   ⚠️  Точное решение не сходится, используем приближенное...")
            # Приближенное решение через SVD
            U, s, Vt = np.linalg.svd(P, full_matrices=False)
            s_inv = 1.0 / s
            P_inv = U @ np.diag(s_inv) @ Vt
            self.B = P_inv / -np.diag(P_inv)
            np.fill_diagonal(self.B, 0)

        # Конвертируем в DataFrame для удобства
        self.B = pd.DataFrame(
            self.B,
            index=self.item_ids,
            columns=self.item_ids
        )

        print("   ✅ EASE модель обучена!")

        # Статистика модели
        self._print_model_stats()

    def _print_model_stats(self):
        """Печать статистики обученной модели"""
        if self.B is None:
            return

        B_values = self.B.values
        np.fill_diagonal(B_values, 0)  # Игнорируем диагональ

        non_zero = np.count_nonzero(B_values)
        total_elements = B_values.size - len(self.item_ids)  # Исключаем диагональ

        print(f"   📊 Статистика модели:")
        print(f"      - Размер матрицы весов: {self.B.shape}")
        print(f"      - Sparsity: {(1 - non_zero / total_elements) * 100:.1f}%")
        print(f"      - Средний вес: {np.mean(np.abs(B_values)):.4f}")
        print(f"      - Максимальный вес: {np.max(np.abs(B_values)):.4f}")

    def recommend(self, user_id, n=10, return_scores=False):
        """
        Рекомендация товаров для пользователя

        Parameters:
        user_id: ID пользователя
        n: количество рекомендаций
        return_scores: возвращать ли scores

        Returns:
        list: список рекомендованных item_id (и scores если return_scores=True)
        """
        if self.B is None:
            raise ValueError("Модель не обучена! Сначала вызовите fit()")

        # Обработка cold start (новый пользователь)
        if user_id not in self.user_ids:
            print(f"   ⚠️  Пользователь {user_id} не найден, возвращаем популярные товары")
            popular_items = self.user_item_matrix.sum(axis=0).sort_values(ascending=False).head(n)
            if return_scores:
                return [(item, score) for item, score in popular_items.items()]
            else:
                return popular_items.index.tolist()

        # Получаем вектор взаимодействий пользователя
        user_interactions = self.user_item_matrix.loc[user_id]

        # Вычисляем scores: X_u * B
        scores = user_interactions @ self.B

        # Убираем уже взаимодействованные товары
        interacted_items = user_interactions[user_interactions > 0].index
        scores[interacted_items] = -np.inf

        # Сортируем по убыванию score
        top_scores = scores.sort_values(ascending=False).head(n)

        if return_scores:
            return [(item, score) for item, score in top_scores.items()]
        else:
            return top_scores.index.tolist()

    def get_similar_items(self, item_id, n=10, return_scores=False):
        """
        Получить похожие товары на основе матрицы EASE

        Parameters:
        item_id: ID товара
        n: количество похожих товаров
        return_scores: возвращать ли scores

        Returns:
        list: список похожих item_id (и scores если return_scores=True)
        """
        if self.B is None:
            raise ValueError("Модель не обучена! Сначала вызовите fit()")

        if item_id not in self.item_ids:
            print(f"   ⚠️  Товар {item_id} не найден")
            return []

        # Получаем веса для данного товара
        item_weights = self.B[item_id].sort_values(ascending=False)

        # Убираем сам товар и фильтруем по положительным весам
        similar_items = item_weights[item_weights.index != item_id]
        similar_items = similar_items[similar_items > 0]  # Только положительные связи

        top_similar = similar_items.head(n)

        if return_scores:
            return [(item, score) for item, score in top_similar.items()]
        else:
            return top_similar.index.tolist()

    def predict_score(self, user_id, item_id):
        """
        Предсказание score для пары пользователь-товар

        Parameters:
        user_id: ID пользователя
        item_id: ID товара

        Returns:
        float: предсказанный score
        """
        if self.B is None:
            raise ValueError("Модель не обучена! Сначала вызовите fit()")

        if user_id not in self.user_ids or item_id not in self.item_ids:
            return 0.0

        user_interactions = self.user_item_matrix.loc[user_id]
        return user_interactions @ self.B[item_id]

    def get_item_relationships(self, item_id, n=10):
        """
        Получить самые сильные связи для товара
        (какие товары больше всего влияют на рекомендации данного товара)

        Parameters:
        item_id: ID товара
        n: количество связей

        Returns:
        DataFrame: связи товара с другими товарами
        """
        if self.B is None:
            raise ValueError("Модель не обучена! Сначала вызовите fit()")

        if item_id not in self.item_ids:
            return pd.DataFrame()

        # Получаем веса для всех товаров
        relationships = self.B[item_id].sort_values(ascending=False)
        relationships = relationships[relationships.index != item_id]  # Убираем сам товар

        return relationships.head(n)

    def get_model_info(self):
        """Информация о модели"""
        if self.B is None:
            return {'model_type': 'EASE', 'status': 'Not trained'}

        B_values = self.B.values
        np.fill_diagonal(B_values, 0)  # Игнорируем диагональ

        # ИСПРАВЛЕННЫЙ РАСЧЕТ SPARSITY
        total_elements = B_values.size - len(self.item_ids)  # Исключаем диагональ
        non_zero = np.count_nonzero(B_values)
        sparsity = (1 - non_zero / total_elements) * 100

        return {
            'model_type': 'EASE',
            'l2_reg': self.l2_reg,
            'normalize': self.normalize,
            'num_users': len(self.user_ids),
            'num_items': len(self.item_ids),
            'matrix_shape': self.B.shape,
            'sparsity': sparsity,  # ✅ ИСПРАВЛЕНО
            'non_zero_elements': non_zero,
            'total_elements': total_elements,
            'avg_weight': np.mean(np.abs(B_values)),
            'max_weight': np.max(np.abs(B_values))
        }




# 🚀 ПРИМЕР ИСПОЛЬЗОВАНИЯ:

def test_ease_recommender(df_clean):
    """
    Тестирование EASE рекомендателя на АКТИВНЫХ пользователях
    """
    print("=" * 60)
    print("🧪 ТЕСТИРОВАНИЕ EASE РЕКОМЕНДАТЕЛЯ")
    print("=" * 60)

    # Инициализация и обучение модели
    recommender = EASERecommender(l2_reg=1, normalize=True)
    recommender.fit(df_clean)

    # НАХОДИМ АКТИВНЫХ ПОЛЬЗОВАТЕЛЕЙ
    user_interaction_counts = df_clean['reviewerID'].value_counts()

    print("📊 СТАТИСТИКА ПОЛЬЗОВАТЕЛЕЙ:")
    print(f"   Всего пользователей: {len(user_interaction_counts)}")
    print(f"   Пользователей с 1 взаимодействием: {(user_interaction_counts == 1).sum()}")
    print(f"   Пользователей с 5+ взаимодействиями: {(user_interaction_counts >= 5).sum()}")
    print(f"   Пользователей с 10+ взаимодействиями: {(user_interaction_counts >= 10).sum()}")

    # ВЫБИРАЕМ РАЗНЫХ ПОЛЬЗОВАТЕЛЕЙ ДЛЯ ТЕСТИРОВАНИЯ
    test_users = {
        'САМЫЙ АКТИВНЫЙ': user_interaction_counts.index[0],  # Топ по взаимодействиям
        'СРЕДНЕАКТИВНЫЙ': user_interaction_counts[user_interaction_counts.between(5, 15)].index[0] if len(user_interaction_counts[user_interaction_counts.between(5, 15)]) > 0 else user_interaction_counts.index[1],
        'С 1 ВЗАИМОДЕЙСТВИЕМ': user_interaction_counts[user_interaction_counts == 1].index[0] if len(user_interaction_counts[user_interaction_counts == 1]) > 0 else user_interaction_counts.index[2]
    }

    # ТЕСТИРУЕМ НА КАЖДОМ ТИПЕ ПОЛЬЗОВАТЕЛЕЙ
    for user_type, test_user in test_users.items():
        print(f"\n{'='*50}")
        print(f"👤 ТЕСТ: {user_type} ПОЛЬЗОВАТЕЛЬ")
        print(f"{'='*50}")

        # Получаем профиль пользователя
        user_interactions = df_clean[df_clean['reviewerID'] == test_user]
        num_interactions = len(user_interactions)
        avg_rating = user_interactions['overall'].mean()
        top_items = user_interactions['asin'].head(3).tolist()

        print(f"   ID: {test_user}")
        print(f"   Взаимодействий: {num_interactions}")
        print(f"   Средний рейтинг: {avg_rating:.2f}")
        print(f"   Топ-3 товара: {top_items}")

        # Получение рекомендаций
        print(f"\n   📋 РЕКОМЕНДАЦИИ:")
        recommendations = recommender.recommend(test_user, n=8, return_scores=True)

        if recommendations:
            for i, (item, score) in enumerate(recommendations, 1):
                # Получаем популярность товара
                item_popularity = len(df_clean[df_clean['asin'] == item])
                print(f"   {i:2d}. {item} | Score: {score:.3f} | Отзывов: {item_popularity}")
        else:
            print("   ❌ Нет рекомендаций")

        # Похожие товары для первого товара из истории пользователя
        if top_items:
            first_item = top_items[0]
            print(f"\n   🔍 ПОХОЖИЕ ТОВАРЫ НА '{first_item}':")
            similar_items = recommender.get_similar_items(first_item, n=5, return_scores=True)

            if similar_items:
                for i, (similar_item, similarity) in enumerate(similar_items, 1):
                    item_popularity = len(df_clean[df_clean['asin'] == similar_item])
                    print(f"   {i:2d}. {similar_item} | Similarity: {similarity:.3f} | Отзывов: {item_popularity}")
            else:
                print("   ❌ Нет похожих товаров")

        # Проверяем разнообразие рекомендаций
        if recommendations:
            rec_items = [item for item, score in recommendations]
            scores = [score for item, score in recommendations if score > 0]

            print(f"\n   📊 СТАТИСТИКА РЕКОМЕНДАЦИЙ:")
            print(f"      - Уникальных товаров: {len(set(rec_items))}")
            print(f"      - Средняя популярность: {np.mean([len(df_clean[df_clean['asin'] == item]) for item in rec_items]):.1f}")
            if scores:
                print(f"      - Max score: {max(scores):.3f}")
                print(f"      - Avg score: {np.mean(scores):.3f}")

    # ДОПОЛНИТЕЛЬНО: ТЕСТ НА СЛУЧАЙНЫХ АКТИВНЫХ ПОЛЬЗОВАТЕЛЯХ
    print(f"\n{'='*50}")
    print("🎲 ТЕСТ НА 3 СЛУЧАЙНЫХ АКТИВНЫХ ПОЛЬЗОВАТЕЛЯХ")
    print(f"{'='*50}")

    active_users = user_interaction_counts[user_interaction_counts >= 3].index
    random_active_users = np.random.choice(active_users, size=min(3, len(active_users)), replace=False)

    for i, random_user in enumerate(random_active_users, 1):
        user_interactions = len(df_clean[df_clean['reviewerID'] == random_user])
        recs = recommender.recommend(random_user, n=5)
        print(f"{i}. {random_user}: {user_interactions} взаимодействий → {len(recs)} рекомендаций")

    # Информация о модели
    print("\n🔍 ИНФОРМАЦИЯ О МОДЕЛИ:")
    model_info = recommender.get_model_info()
    for key, value in model_info.items():
        print(f"   {key}: {value}")

    return recommender


In [ ]:
import numpy as np
from collections import defaultdict
import math

class RecommendationMetrics:
    """
    Класс для расчета метрик качества рекомендаций
    """

    def __init__(self, test_interactions, catalog_size):
        self.test_interactions = test_interactions  # DataFrame с тестовыми взаимодействиями
        self.catalog_size = catalog_size
        self._prepare_ground_truth()

    def _prepare_ground_truth(self):
        """Подготовка ground truth данных"""
        self.ground_truth = defaultdict(set)
        for _, row in self.test_interactions.iterrows():
            self.ground_truth[row['reviewerID']].add(row['asin'])

    def precision_at_k(self, recommendations, k=10):
        """
        Precision@K - точность среди топ-K рекомендаций
        """
        precisions = []

        for user_id, recs in recommendations.items():
            if user_id not in self.ground_truth:
                continue

            true_positives = len(set(recs[:k]) & self.ground_truth[user_id])
            precision = true_positives / k
            precisions.append(precision)

        return np.mean(precisions) if precisions else 0.0

    def recall_at_k(self, recommendations, k=10):
        """
        Recall@K - полнота среди топ-K рекомендаций
        """
        recalls = []

        for user_id, recs in recommendations.items():
            if user_id not in self.ground_truth:
                continue

            true_positives = len(set(recs[:k]) & self.ground_truth[user_id])
            relevant_items = len(self.ground_truth[user_id])
            recall = true_positives / relevant_items if relevant_items > 0 else 0
            recalls.append(recall)

        return np.mean(recalls) if recalls else 0.0

    def ndcg_at_k(self, recommendations, k=10):
        """
        NDCG@K - нормализованный дисконтированный кумулятивный выигрыш
        Учитывает порядок рекомендаций
        """
        ndcgs = []

        for user_id, recs in recommendations.items():
            if user_id not in self.ground_truth:
                continue

            dcg = 0.0
            for i, item in enumerate(recs[:k]):
                if item in self.ground_truth[user_id]:
                    dcg += 1 / math.log2(i + 2)  # i+2 потому что индекс с 0

            # Ideal DCG
            ideal_dcg = 0.0
            min_len = min(k, len(self.ground_truth[user_id]))
            for i in range(min_len):
                ideal_dcg += 1 / math.log2(i + 2)

            ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0
            ndcgs.append(ndcg)

        return np.mean(ndcgs) if ndcgs else 0.0

    def catalog_coverage(self, recommendations, k=10):
        """
        Покрытие каталога - какой процент товаров рекомендован
        """
        all_recommended_items = set()

        for recs in recommendations.values():
            all_recommended_items.update(recs[:k])

        return len(all_recommended_items) / self.catalog_size

    def novelty(self, recommendations, item_popularity, k=10):
        """
        Новизна - рекомендует ли система непопулярные товары
        """
        novelties = []

        for recs in recommendations.values():
            rec_novelty = 0.0
            for item in recs[:k]:
                # Чем менее популярен товар, тем выше новизна
                popularity = item_popularity.get(item, 0)
                rec_novelty += -math.log2((popularity + 1) / (self.catalog_size + 1))

            novelties.append(rec_novelty / k)

        return np.mean(novelties) if novelties else 0.0

    def diversity(self, recommendations, item_similarity_matrix, k=10):
        """
        Разнообразие - насколько рекомендации отличаются друг от друга
        """
        diversities = []

        for recs in recommendations.values():
            rec_diversity = 0.0
            count = 0

            for i in range(len(recs[:k])):
                for j in range(i + 1, len(recs[:k])):
                    item_i, item_j = recs[i], recs[j]
                    similarity = item_similarity_matrix.get((item_i, item_j), 0)
                    rec_diversity += 1 - similarity
                    count += 1

            diversities.append(rec_diversity / count if count > 0 else 0)

        return np.mean(diversities) if diversities else 0.0

    def calculate_all_metrics(self, recommendations, item_popularity, item_similarity_matrix, k_values=[5, 10]):
        """
        Расчет всех метрик для разных K
        """
        results = {}

        for k in k_values:
            results[f'Precision@{k}'] = self.precision_at_k(recommendations, k)
            results[f'Recall@{k}'] = self.recall_at_k(recommendations, k)
            results[f'NDCG@{k}'] = self.ndcg_at_k(recommendations, k)

        results['Catalog_Coverage'] = self.catalog_coverage(recommendations, k=10)
        results['Novelty'] = self.novelty(recommendations, item_popularity, k=10)
        results['Diversity'] = self.diversity(recommendations, item_similarity_matrix, k=10)

        return results

### **Выполнение задания**

In [ ]:
df = load_amazon_dataset()

In [ ]:
profile = comprehensive_data_profile(df)
profile

In [ ]:
full_eda_analysis()

In [ ]:
df_clean = data_cleaning(df, keep_text_data=False, min_user_interactions=3, min_item_reviews=3, min_year=2013)

In [ ]:
df_clean.head(6)

In [ ]:
test_mostpop_recommender(df_clean)

In [ ]:
test_itemknn_recommender(df_clean)

In [ ]:
test_ease_recommender(df_clean)

In [ ]:
evaluate_all_models(df_clean)